In [512]:

import numpy as np
import plotly
import plotly.graph_objs as go
import torch

import torch.nn as nn

from plotly.graph_objs import *

In [513]:
from scipy.special import iv

In [573]:
embmat = torch.load('../data/distilbert_embedding_matrix.pt')

In [577]:
embmat.norm(dim=-1).std()

tensor(0.2599)

In [514]:
#just a sphere
size = 200
theta = np.linspace(0,2*np.pi,size)
phi = np.linspace(0,np.pi,size)
x = np.outer(np.cos(theta),np.sin(phi))
y = np.outer(np.sin(theta),np.sin(phi))
z = np.outer(np.ones(size),np.cos(phi))  # note this is 2d now

data = go.Surface(
        x=x,
        y=y,
        z=z,
#  surfacecolor=r,
    )
layout = go.Layout(
    title='sphere',
    autosize=False,
    width=1000,
    height=1000,
)
fig = go.Figure(data=[data], layout=layout)


In [289]:
plotly.offline.plot(fig, filename='/home/tucker/Downloads/sphere.html')

'/home/tucker/Downloads/sphere.html'

In [515]:
def C(kappa, p):
  num = kappa ** (0.5*p - 1)
  denom = (2 * np.pi) ** (0.5*p) * iv(0.5*p - 1, kappa)
  normalization_const = num/denom
  return normalization_const


In [516]:
def vmf(x, mu, kappa):
    """Adapted from https://github.com/lmc2179/von_mises_fisher"""
    normalization_const = C(kappa, 100)
    likelihood = np.exp(kappa * (x@mu))
    return normalization_const * likelihood


In [517]:
def vmf2(x, mu, kappa):
    """Adapted from https://github.com/lmc2179/von_mises_fisher"""
    normalization_const = C(kappa, 100)
    likelihood = np.exp((x@mu))
    return normalization_const * likelihood


In [518]:
def vmf_reg2(x, mu, kappa):
    """Adapted from https://github.com/lmc2179/von_mises_fisher"""
    normalization_const = C(kappa, mu.shape[-1])
    likelihood = np.exp(.1*(x@mu))
    return normalization_const * likelihood


In [592]:
def vmf_reg12(x, mu, kappa, r1, r2):
  """Adapted from https://github.com/lmc2179/von_mises_fisher"""
  normalization_const = C(kappa, mu.shape[-1])
  likelihood = np.exp(r2*(x@mu))
  reg_term = np.exp(-1*r1 * kappa)
  return normalization_const * likelihood * reg_term


In [520]:
def vmf_simp(x, mu, kappa):
    """Adapted from https://github.com/lmc2179/von_mises_fisher"""
    normalization_const = kappa
    likelihood = np.exp((x@mu))
    return normalization_const * likelihood


In [522]:
def vmf_simp_loss(out, targets):
  normalization_const = out.norm(dim=-1)
  # batchwise matrix multiply
  likelihood = out.unsqueeze(1)@targets.view(-1,targets.shape[-1], 1)
  return (-torch.log(normalization_const) - likelihood.squeeze()).mean()

In [523]:
emb = torch.tensor([[1,1,3.0],
                    [1,1,1],
                     [-1,0,1],
                    [1,0,1],]
)
b  = torch.tensor([[1,1,1.0], [2,1,1.0]])
labels = torch.tensor([1,3])                   

In [524]:
def vmf_nearest_neighbor(out, labels, embeddings):
  print(out.shape, labels.shape, embeddings.shape)
  pass

In [525]:
vmf_nearest_neighbor(b, labels, emb)

torch.Size([2, 3]) torch.Size([2]) torch.Size([4, 3])


In [526]:
vmf_simp_loss(a,b)

tensor(-5.3741)

In [527]:
def vmf_reg1(x, mu, kappa):
    """Adapted from https://github.com/lmc2179/von_mises_fisher"""
    normalization_const = C(kappa, mu.shape[-1])
    likelihood = np.exp((x@mu))
    reg_term = np.exp(-.02 * kappa)
    return normalization_const * likelihood * reg_term


In [529]:
def vmf_normed(out, target):
  return -1* vmf(out, target, np.linalg.norm(out, axis=-1))

In [530]:
def vmf_normed_2(out, target):
  return -1* vmf2(out, target, np.linalg.norm(out, axis=-1))

In [531]:
def vmf_normed_simp(out, target):
  return -1* vmf_simp(out, target, np.linalg.norm(out, axis=-1))

In [532]:
def vmf_normed_reg1(out, target):
  return -1* vmf_reg1(out, target, np.linalg.norm(out, axis=-1))

In [533]:
def vmf_normed_reg2(out, target):
  return -1* vmf_reg2(out, target, np.linalg.norm(out, axis=-1))

In [593]:
def make_vmf_normed_reg12(r1, r2):
  def vmf_normed_reg12(out, target):
    return -1 * vmf_reg12(out, target, np.linalg.norm(out, axis=-1), r1, r2)
  return vmf_normed_reg12

In [535]:
x = np.array([[[3,0,0,], [3,0,0.0]], [[3,0,0,], [3,0,0.0]]])
mu = np.array([1,0,0.0])
x@mu

array([[3., 3.],
       [3., 3.]])

In [536]:
vmf(x, mu, 2)

array([[1.66979802e+40, 1.66979802e+40],
       [1.66979802e+40, 1.66979802e+40]])

In [537]:
# wanna know- is this gonna get too big? yes

def NLLvMF(output, target_embedding):
  return -torch.log(C(output.norm(dim=1), output.size(-1))) - output@target_embedding
def NLLvMF(output, target_embedding, kappa):
  return -torch.log(C(kappa, output.size(-1))) - output@target_embedding
def NIP(output, target_embedding, kappa):
  return - output@target_embedding

In [538]:
a = torch.tensor([1,1,1,1]).float()*2
b = torch.tensor([1,1,1,1]).float()

In [539]:
NLLvMF(a, b, torch.tensor([1.0]))

tensor([-4.8949])

In [540]:
vs = np.stack([x,y,z], -1)

ValueError: all input arrays must have the same shape

In [541]:
v = torch.tensor(vs).reshape(-1,3).float()
r = NIP(v, torch.tensor([1.,1,1]), torch.tensor([.0001])).reshape(200,200)

NameError: name 'vs' is not defined

In [542]:
v.reshape(-1,3).norm(dim=1)

NameError: name 'v' is not defined

In [543]:
import numpy as np


In [583]:
torch.nn.Embedding(100,768).weight.data.norm(dim=-1).

tensor([25.6711, 28.1267, 28.1889, 27.9041, 27.6386, 26.8485, 26.7191, 27.7992,
        27.9356, 28.9243, 28.4940, 26.9299, 26.9750, 27.2730, 26.7266, 27.8710,
        26.3348, 27.3194, 26.8617, 27.8161, 27.6180, 26.1009, 26.9545, 28.1553,
        26.8330, 28.5992, 27.1094, 28.5591, 27.7864, 27.7719, 26.6565, 26.9131,
        28.3324, 28.8200, 27.4723, 28.8258, 27.9311, 27.2504, 26.7574, 27.6124,
        27.7960, 27.0214, 27.9871, 27.2258, 28.0194, 26.5476, 27.4728, 27.5441,
        27.6225, 28.2002, 27.7051, 27.1864, 27.2724, 28.3816, 27.7944, 28.2787,
        28.1877, 28.3125, 26.9053, 27.8714, 27.7694, 28.5095, 26.9804, 29.1247,
        26.7587, 28.2815, 27.5244, 27.7200, 27.3084, 28.5066, 27.3882, 27.7525,
        27.3525, 27.3271, 26.5434, 27.6291, 27.9861, 28.5442, 26.6987, 27.6752,
        28.4884, 29.8203, 28.4597, 27.6188, 28.0711, 28.8631, 28.8927, 28.7972,
        26.1748, 27.7127, 27.2244, 28.1270, 26.8756, 26.1398, 27.4298, 28.0768,
        27.2245, 28.2989, 27.1706, 28.53

In [628]:
theta_x = np.linspace(-2*np.pi, 2*np.pi, 500)
r = np.linspace(-5.8,5.8,500)

In [629]:
def convert_spherical_to_cartesian_2d(rho, theta):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

In [630]:
points = []
for theta in theta_x:
  for rad in r:
    x,y = convert_spherical_to_cartesian_2d(rad, theta)
    points.append(np.array([x, y]))

In [631]:
points = np.stack(points).reshape(len(theta_x),len(r),2)

In [632]:
target_rad = 1.6
target_theta = 0.0
target_point = np.array(convert_spherical_to_cartesian_2d(target_rad, target_theta))

In [633]:
def MSE(out, target):
  # out is a nXnX2 and target is 2
  dif_sq = (out - target)**2
  s = dif_sq.sum(-1)
  return s

In [634]:
def sq_MSE(out, target):
  return np.sqrt(MSE(out, target))

In [635]:
(points@target_point).shape

(500, 500)

In [636]:
def inner_product(out, target):
  print(out.shape, target.shape)
  return -1*points@target_point

In [637]:
def cosine_similarity(out, target):
  return -1*points@target_point/np.linalg.norm(points, axis=-1)

In [638]:
mse_points = MSE(points, target_point)

In [639]:
mse_points.shape

(500, 500)

In [640]:
np.gradient(mse_points)[0].shape

(500, 500)

In [647]:
def plot(points, target_point, fn):
  dist_points = fn(points, target_point)
  grads_points = np.abs(np.gradient(dist_points)[1])
  data = go.Surface(
          x=r,
          y=theta_x,
          z=dist_points,
    surfacecolor=grads_points,
 showscale=True,
      )
  layout = go.Layout(
      title=str(fn).split()[1] + " colored by grad wrt norm",
      autosize=False,
      width=1200,
      height=1000,
    scene={'xaxis_title':'norm',
           'yaxis_title':'theta',
           'aspectmode':'cube',
           'aspectratio': {'x':1,'y':3}}
  )
  p = np.unravel_index(dist_points.argmin(), dist_points.shape)
  minp = {'type':'scatter3d',
          'x':[r[p[1]]],
          'y':[theta_x[p[0]]],
          'z':[dist_points[p]],}
  fig = go.Figure(data=[data, minp], layout=layout)
  plotly.offline.plot(fig, filename='/home/tucker/Downloads/sphere.html')


  print('min_index', p)
  print('min point', points[p])
  print('min r the', r[p[1]], theta_x[p[0]])
  print('min dist:', dist_points[p])
  print('min grad:', grads_points.min())
  print('max grad:', grads_points.max())  

In [558]:
plot(points, target_point, MSE)

min_index (250, 155)
min point [1.00158336 0.00630583]
min r the 1.0016032064128255 0.006295776860901103
min dist: 4.227049193364066e-05
min grad: 3.177194153880514e-07
max grad: 0.12591158614034548


In [559]:
plot(points, target_point, sq_MSE)

min_index (250, 155)
min point [1.00158336 0.00630583]
min r the 1.0016032064128255 0.006295776860901103
min dist: 0.006501576111501015
min grad: 1.585451427921214e-07
max grad: 0.012591220998056762


In [560]:
plot(points, target_point, inner_product)

(500, 500, 2) (2,)


min_index (249, 499)
min point [ 4.99990091 -0.03147868]
min r the 5.0 -0.006295776860901547
min dist: -4.999900908311601
min grad: 1.588597076860026e-07
max grad: 0.06295579307017342


In [561]:
plot(points, target_point, cosine_similarity)

min_index (0, 0)
min point [8.00000000e-01 9.79717439e-17]
min r the -0.8 -3.141592653589793
min dist: -1.0
min grad: 7.927099413518324e-05
max grad: 0.012591158614034686


In [562]:
plot_grad(points, target_point, cosine_similarity)

min_index (125, 80)
min point [ 0.00040878 -0.12985908]
min r the 0.12985971943887775 -1.567648438364446
min dist: -0.012591158614034686


In [563]:
plot(points, target_point, vmf_normed)

min_index (249, 499)
min point [ 4.99990091 -0.03147868]
min r the 5.0 -0.006295776860901547
min dist: -2.6823016447673724e+48
min grad: 1.0268145939661264e+24
max grad: 1.017836965078111e+47


In [564]:
# This is the unregularized version from the paper (it excluded the kappa term (norm of e) in the exponent/innerproduct term
plot(points, target_point, vmf_normed_2)

min_index (249, 499)
min point [ 4.99990091 -0.03147868]
min r the 5.0 -0.006295776860901547
min dist: -5.530827553339808e+39
min grad: 6.694733599657165e+30
max grad: 9.209360494997154e+37


In [565]:
plot_grad(points, target_point, vmf_normed_2)

min_index (215, 499)
min point [ 4.53559552 -2.10437004]
min r the 5.0 -0.4344086034021961
min dist: -9.209360494997154e+37


In [566]:
plot(points, target_point, vmf_normed_simp)

min_index (249, 499)
min point [ 4.99990091 -0.03147868]
min r the 5.0 -0.006295776860901547
min dist: -741.9922666034148
min grad: 3.1772510013586275e-10
max grad: 12.354885777490182


In [567]:
plot(points, target_point, vmf_normed_reg1)

min_index (249, 499)
min point [ 4.99990091 -0.03147868]
min r the 5.0 -0.006295776860901547
min dist: -0.7845398323424346
min grad: 1.4121869774326207e-08
max grad: 0.013063343720350146


In [568]:
plot(points, target_point, vmf_normed_reg2)

min_index (249, 86)
min point [ 0.19959524 -0.00125662]
min r the 0.19959919839679352 -0.006295776860901547
min dist: -0.16075837604904103
min grad: 2.5278717452614785e-09
max grad: 0.00018561328788017917


In [648]:
plot(points, target_point, make_vmf_normed_reg12(1, 1))

min_index (0, 315)
min point [1.52264529e+00 3.72940536e-16]
min r the 1.5226452905811625 -6.283185307179586
min dist: -0.237725203688139
min grad: 6.816203498781849e-11
max grad: 0.008846192947442016
